In [ ]:
import scipy.linalg
!pip3 install git+https://github.com/Centre-automatique-et-systemes/lena.git gwpy &> /dev/null
!pip3 install git+https://github.com/aliutkus/torchinterp1d.git gwpy &> /dev/null

In [1]:
import sys ; sys.path.append('../')
import scipy.signal as sig
import torch.optim as optim
import torch
import seaborn as sb
import pytorch_lightning as pl
import numpy as np

from sklearn.model_selection import train_test_split
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

from learn_KKL.luenberger_observer import LuenbergerObserver
from learn_KKL.system import pendulum
from learn_KKL.learner import Learner

sb.set_style('whitegrid')

In [4]:
# Set up the system
system = pendulum()

In [5]:
# Instantiate the observer
#D = torch.diag_embed(torch.tensor([-.1, -.2, -.3]))
#F = np.ones((3,1))
#Canonical form D
#_, coeffs = sig.bessel(3, 0.03, analog=True)
#D = np.polynomial.polynomial.polycompanion(np.flip(coeffs)).T
#F = np.array([[1.], [1.], [1.]])

#Near-diagonal form D
#_,p,_ = sig.bessel(3, 0.2, analog=True, output='zpk')
#D = np.array([[p[1].real, 0, 0],[0, p[0].real, p[0].imag],[0,p[2].imag, p[2].real]])
#F = np.ones((3,1))

#Another near-diagonal form
#_,p,_ = sig.bessel(3, 0.2, analog=True, output='zpk')
#D = np.array([[p[1].real, 0, 0],[0, 0, 1],[0,- p[0] * p[2], 2 * p[2].real]]).real
#F = np.array([[1.], [1.], [1.]])

observer = LuenbergerObserver(dim_x=2, dim_y=1, method='Supervised', wc=0.1,
                              recon_lambda=0.8)
observer.set_dynamics(system)
# Generate (x_i, z_i) data by running system backward, then system + observer
# forward in time
data = observer.generate_data_svl(np.array([[-np.pi, np.pi], [-1., 1.]]), 7200,
                                  method='uniform')
data, val_data = train_test_split(data, test_size=0.3, shuffle=True)

# Train the forward transformation using pytorch-lightning and the learner class
# Options for training
trainer_options={'max_epochs': 15}
optimizer_options = {'weight_decay': 1e-6}
scheduler_options = {'mode': 'min', 'factor': 0.1, 'patience': 3,
                     'threshold': 1e-4, 'verbose': True}
stopper = pl.callbacks.early_stopping.EarlyStopping(
    monitor='val_loss', min_delta=5e-4, patience=3, verbose=False, mode='min')
# Instantiate learner
learner_T = Learner(observer=observer, system=system, training_data=data,
                    validation_data=val_data, method='T', batch_size=10,
                    lr=1e-3, optimizer=optim.Adam,
                    optimizer_options=optimizer_options,
                    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
                    scheduler_options=scheduler_options)
# Define logger and checkpointing
logger = TensorBoardLogger(save_dir=learner_T.results_folder + '/tb_logs')
checkpoint_callback = ModelCheckpoint(monitor='val_loss')
trainer = pl.Trainer(
    callbacks=[stopper, checkpoint_callback], **trainer_options, logger=logger,
    log_every_n_steps=1, check_val_every_n_epoch=3)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Results saved in in /Users/fdm/Documents/GitHub/learn_observe_KKL/jupyter_notebooks/runs/pendulum/Supervised/T/exp_0


In [6]:
# To see logger in tensorboard, copy the following output name_of_folder
print(f'Logs stored in {learner_T.results_folder}/tb_logs')
# which should be similar to jupyter_notebooks/runs/method/exp_0/tb_logs/
# Then type this in terminal:
# tensorboard --logdir=name_of_folder --port=8080

# Train and save results
trainer.fit(learner_T)
learner_T.save_results(limits=np.array([[-np.pi, np.pi], [-1., 1.]]),
                       nb_trajs=10, tsim=(0, 60), dt=1e-2,
                       checkpoint_path=checkpoint_callback.best_model_path)


  | Name  | Type               | Params
---------------------------------------------
0 | model | LuenbergerObserver | 26.1 K
---------------------------------------------
26.1 K    Trainable params
0         Non-trainable params
26.1 K    Total params
0.104     Total estimated model params size (MB)


Logs stored in /Users/fdm/Documents/GitHub/learn_observe_KKL/jupyter_notebooks/runs/pendulum/Supervised/T/exp_0/tb_logs


Validation sanity check: 0it [00:00, ?it/s]

/Users/fdm/opt/miniconda3/envs/numerical_KKL/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:440: UserWarning: Your `val_dataloader` has `shuffle=True`,it is strongly recommended that you turn this off for val/test/predict dataloaders.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Epoch    10: reducing learning rate of group 0 to 1.0000e-04.


Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

0

In [7]:
# Train the inverse transformation using pytorch-lightning and the learner class
# Options for training
trainer_options={'max_epochs': 20}
optimizer_options = {'weight_decay': 1e-8}
scheduler_options = {'mode': 'min', 'factor': 0.1, 'patience': 3,
                     'threshold': 1e-4, 'verbose': True}
stopper = pl.callbacks.early_stopping.EarlyStopping(
    monitor='val_loss', min_delta=5e-4, patience=3, verbose=False, mode='min')
# Instantiate learner
learner_T_star = Learner(observer=observer, system=system, training_data=data,
                         validation_data=val_data, method='T_star',
                         batch_size=10, lr=5e-4, optimizer=optim.Adam,
                         optimizer_options=optimizer_options,
                         scheduler=optim.lr_scheduler.ReduceLROnPlateau,
                         scheduler_options=scheduler_options)
# Define logger and checkpointing
logger = TensorBoardLogger(save_dir=learner_T_star.results_folder + '/tb_logs')
checkpoint_callback = ModelCheckpoint(monitor='val_loss')
trainer = pl.Trainer(
    callbacks=[stopper, checkpoint_callback], **trainer_options, logger=logger,
    log_every_n_steps=1, check_val_every_n_epoch=3)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Results saved in in /Users/fdm/Documents/GitHub/learn_observe_KKL/jupyter_notebooks/runs/pendulum/Supervised/T_star/exp_0


In [8]:
# To see logger in tensorboard, copy the following output name_of_folder
print(f'Logs stored in {learner_T_star.results_folder}/tb_logs')
# which should be similar to jupyter_notebooks/runs/method/exp_0/tb_logs/
# Then type this in terminal:
# tensorboard --logdir=name_of_folder --port=8080

# Train and save results
trainer.fit(learner_T_star)
learner_T_star.save_results(limits=np.array([[-np.pi, np.pi], [-1., 1.]]),
                            nb_trajs=10, tsim=(0, 60), dt=1e-2,
                            checkpoint_path=checkpoint_callback.best_model_path)



  | Name  | Type               | Params
---------------------------------------------
0 | model | LuenbergerObserver | 26.1 K
---------------------------------------------
26.1 K    Trainable params
0         Non-trainable params
26.1 K    Total params
0.104     Total estimated model params size (MB)


Logs stored in /Users/fdm/Documents/GitHub/learn_observe_KKL/jupyter_notebooks/runs/pendulum/Supervised/T_star/exp_0/tb_logs


Validation sanity check: 0it [00:00, ?it/s]

/Users/fdm/opt/miniconda3/envs/numerical_KKL/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:440: UserWarning: Your `val_dataloader` has `shuffle=True`,it is strongly recommended that you turn this off for val/test/predict dataloaders.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

...

Epoch    17: reducing learning rate of group 0 to 5.0000e-06.


Validating: 0it [00:00, ?it/s]

No handles with labels found to put in legend.


Shape of mesh for evaluation: torch.Size([10000, 5])


No handles with labels found to put in legend.
No handles with labels found to put in legend.
